# Integrating LangChain Tools with the AG2 Framework

In this tutorial, we demonstrate how to integrate [LangChain Tools](https://python.langchain.com/v0.1/docs/modules/tools) into the AG2 framework. This process enables smooth interoperability between the two systems, allowing developers to leverage LangChain's powerful tools within AG2's flexible agent-based architecture. By the end of this guide, you will understand how to configure agents, convert LangChain tools for use in AG2, and validate the integration with a practical example.


## Installation
To integrate LangChain tools into the AG2 framework, install the required dependencies:

```bash
pip install ag2[interop-langchain]
```

Additionally, this notebook uses LangChain's [Wikipedia Tool](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.wikipedia.tool.WikipediaQueryRun.html), which requires the `wikipedia` package. Install it with:

```bash
pip install wikipedia
```

## Imports

Import necessary modules and tools.
- `os`: For accessing environment variables.
- `WikipediaQueryRun` and `WikipediaAPIWrapper`: Tools for querying Wikipedia.
- `AssistantAgent` and `UserProxyAgent`: Agents that facilitate communication in the AG2 framework.
- `LangchainInteroperability`: A bridge for integrating LangChain tools with the AG2 framework.

In [1]:
import os

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from autogen import AssistantAgent, UserProxyAgent
from autogen.interoperability.langchain import LangchainInteroperability

## Agent Configuration

Configure the agents for the interaction.
- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [2]:
config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

chatbot = AssistantAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
)

## Tool Integration

- Initialize and register the LangChain tool with AG2.
- `WikipediaAPIWrapper`: Configured to fetch the top 1 result from Wikipedia with a maximum of 1000 characters per document.
- `WikipediaQueryRun`: A LangChain tool that executes Wikipedia queries.
- `LangchainInteroperability`: Converts the LangChain tool into a format compatible with the AG2 framework.
- `ag2_tool.register_for_execution(user_proxy)`: Registers the tool for use by the user_proxy agent.
- `ag2_tool.register_for_llm(chatbot)`: Registers the tool for integration with the chatbot agent.


In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
langchain_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

langchain_interop = LangchainInteroperability()
ag2_tool = langchain_interop.convert_tool(langchain_tool)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

## Wikipedia Browsing

- `user_proxy` queries the `chatbot`, which uses a Wikipedia tool to retrieve information.
- The `chatbot` identifies the query's intent and fetches a summary from Wikipedia.
- Tool execution returns a concise response from the relevant Wikipedia page.

In [ ]:
message = "Tell me about the history of the United States"
user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=2)